## Importation des librairies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing
import os
import seaborn as sns

from datetime import datetime
from sklearn.model_selection import train_test_split
import datetime

from keras.layers import Dense,Dropout,SimpleRNN,LSTM, GRU, Bidirectional
from keras.models import Sequential
from sklearn.metrics import r2_score
from keras import optimizers

pd.set_option('max_column', None)

## Création d'une liste des idenfitiants présent dans le dossier dfcomp_vent et d'une liste contenant tout les URL csv

In [ ]:
PATH = 'Projet-EDF/dfcomp_vent'
URL_ALL = []
liste_id = []
for name in os.listdir(PATH):
    liste_id.append(name.split('_')[1].split('.')[0])
    URL_ALL.append(os.path.join(PATH, name))
print('Le dossier contient ',len(URL_ALL),'stations différentes avec la température ET le vent')
print('_____________________________________________')
print('Un exemple des 5 premiers élements de liste_id :',liste_id[:5])
print('_____________________________________________')
print('Un exemple des 5 premiers élements de URL_ALL :\n',URL_ALL[:5])


# Preprocessing sur les df comp

L'idée ici est de concatener tous les fichiers csv du type comp_X où X est l'identifiant de la station. Chacun de ces csv contient le relevé en température et en vent de la station X à pas de temps horaire sur une durée de quasi 10 mois. Ils sont accompagnées de leur données de référence ( = "synop").

### Exemple d'un df_comp 

In [ ]:
index = np.random.randint(0,len(URL_ALL)) 
print('Exemple du dataframe pour la station avec comme identifiant : ', liste_id[index])
pd.read_csv(URL_ALL[index],  sep=';', index_col = 'Unnamed: 0')

In [ ]:
len(URL_ALL)

Plusieurs remarques avant de concatener tous les dataframes :
- Ici, les seules colonnes qui nous intéressent sont les dates (l'index) ; la température météociel et le vent météociel !
- Pour une première approche, on va seulement considérer les dataframe avec aucunes valeurs manquantes sur les données météociel (il reste plus que la moitié des stations)
- On va arrondir les données de vent synop à la 3ème décimales

In [ ]:
def concatenation_dfcomp(liste_url):
    liste_df = [] ### Initialisation de la liste qui permettra de réaliser la concatenation à la fin
    
    columns_to_drop = ['date','temperature_synop','vent_synop','ind','annee','mois','jour','heure','difference_temperature','difference_vent'] ### les colonnes à enlever
    compt_na = 0 ### Un compteur qui va comptabiliser les stations qui ne seront pas concaténer car présentant des valeurs manquantes
    liste_sans_na_id = [] ### Liste qui retriendra tout les identifiants des df qui seront concaténés
    
    for index, url in enumerate(URL_ALL):
        
        ### Mise en forme du df en accord avec les points précédents
        df = pd.read_csv(url, sep=';', index_col = 'Unnamed: 0')
        id = int(liste_id[index])
        df = df.rename(columns = {'Unnamed: 0':'Datetime'}).drop(columns_to_drop, axis=1)
        df.index = pd.to_datetime(df.index)
        df['vent_meteociel'] = df['vent_meteociel'].apply(lambda x : round(x,2))
        
        ### Variable contenant le nombre de valeurs manquantes sur la colonne température météociel et température vent
        isna = df.isna().sum().tolist()
        
        ### On ajoute pour chaque dataframe une colonne de son identifiant, ce qui permettra de le repérer par la suite
        id_serie = [id] * len(df)
        df['id'] = id_serie

        ### On regarde si isna == [0, 0] en d'autres termes, si le df présente aucune valeurs manquantes sur les données météociel
        if isna != [0, 0]:
            compt_na = compt_na + 1
        
        else:
            liste_sans_na_id.append(id)
            liste_df.append(df)
    
    ### Concaténation des df
    df_tot = pd.concat([df for df in liste_df]).reset_index() ### On reset_index car l'index est notre pas de temps et par la suite, c'est mieux d'éviter d'avoir notre pas de temps en index
    
    return [df_tot, compt_na, liste_sans_na_id]

In [ ]:
df_tot, compt_na, liste_id_sans_na = concatenation_dfcomp(URL_ALL)
print('Le df concatené a un shape de ', df_tot.shape)
print('_____________________________________________')
print('Le nombre de stations avec des valeurs manquantes est ',compt_na,', ce qui fait ',round(compt_na/len(URL_ALL)*100,3),'% de df non utilisé')
print('_____________________________________________')
print('Exemple dun sample du dftot')
df_tot.sample(10)

# Preprocessing de df map

df_map est la base de données qui fournit les informations relatives à chaque stations (ville, département, altitude, coordonnées GPS ...). Du preprocessing s'impose afin de la mettre dans une forme acceptable. 

In [ ]:
df_map = pd.read_csv('Projet-EDF/Stations Meteociel 2 - Stations.csv')
df_map = df_map[['id Station', 'Coordonnées GPS']]

In [ ]:
import math

# Convertisseur de coordonées GPS en Degré

def gps2deg(coord): 
    coord1 = str(coord).split('°')
    print(coord1)
    d = int(coord1[0])
    m = int(coord1[1][0:2])
    s = int(coord1[1][4:5])
    dd = d + m/60 + s/3600
    rd = dd/180*np.pi
    return rd/np.pi*180

In [ ]:
# L'idée ici est de convertir toutes les coordonées GPS en degré pour la suite (le problème est de prendre en compte la différence Nord/Sud et Est/Ouest)

df_split_coord = df_map['Coordonnées GPS'].str.split(expand = True)
df_map['lat_mc'] = df_split_coord.loc[:,0].apply(gps2deg)
df_map['long_mc'] = df_split_coord.loc[:,2].apply(gps2deg)
df_split_coord['S?'] = (df_split_coord.loc[:,1] == 'S') | (df_split_coord.loc[:,1] == 'South')
df_split_coord['O?'] = (df_split_coord.loc[:,3] == 'O') | (df_split_coord.loc[:,3] == 'Ouest')
for k in range(len(df_meteociel)):
    if df_split_coord.loc[k,'S?']:
        df_map.loc[k,'lat_mc'] =   - df_map.loc[k,'lat_mc']
    elif df_split_coord.loc[k,'O?']:
        df_map.loc[k,'long_mc'] =   - df_map.loc[k,'long_mc']

## Objectif 
L'objectif maintenant est de mettre en forme le dataframe précedent telle qu'il soit alimenté dans le futur algorithme d'apprentissage automatique.
On souhaite avoir un dataframe de la façon suivante : 
- chaque ligne correspond à un instant t, donc ici une date avec l'année ; le mois ; le jour et l'heure
- chaque colonne correspond à une valeur d'une température ou de vent d'une station spécifique, du type temp_355 qui correspond à la température de la station 355

Pour ce faire, on va utiliser un outil de panda qui s'appel "pivot" qui est résumé dans l'image suivante :

<img src="https://tse3.mm.bing.net/th?id=OIP.8RrnZvQQDumdLm9A23C7HwHaDz&pid=Api">

Il faut donc ajouter le nom des colonnes dans le dataframe original pour après appliquer le pivot !

In [ ]:
def ajout_colonne_nom(df, liste_id):
    ### date unique est une variable qui sauvegarde toutes les dates différentes du df
    date_unique = df_tot['index'].unique()
    
    liste_nom_colonne_temp = []
    liste_nom_colonne_vent = []
    
    for ident in liste_id:
        N = len(df[df['id']== int(ident)])
        
        for index in range(N):
            liste_nom_colonne_temp.append('temp_{}'.format(ident))
            liste_nom_colonne_vent.append('vent_{}'.format(ident))
    
    df['nom_temp'] = liste_nom_colonne_temp
    df['nom_vent'] = liste_nom_colonne_vent
    
    return df

In [ ]:
df = ajout_colonne_nom(df_tot, liste_id_sans_na)
df.sample(10)

In [ ]:
def double_pivot(df):
    
    df_temp = df.pivot(index = 'index', columns = 'nom_temp', values = 'temperature_meteociel')
    df_vent = df.pivot(index = 'index', columns = 'nom_vent', values = 'vent_meteociel')
    
    df_final_ml = pd.merge(df_temp, df_vent, how='outer', on='index')
    
    return df_final_ml

In [ ]:
df_final_ml = double_pivot(df)
print('Shape de df_final_ml :',df_final_ml.shape)

Ainsi, le shape du dataframe est cohérent. En effet : 
- il y a 360 - 189 = 171 stations dans notre dataframe. Pour chaque station, on a la température et le vent d'ou 171*2 = 342 colonnes

## 2ème temps : EDA sur le df précedemment crée
Maintenant qu'on a obtenu le df souhaité, il faut d'abord se préocuper des valeurs manquantes qui ont l'air bien présente
On réalise un peu d'EDA pour voir l'importance des NaN !

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df_final_ml.isna(), cbar=False)

Le blanc représente les valeurs manquantes au sein du dataframe, déjà quel est son pourcentage ?

In [ ]:
pourcentage_missing_value = df_final_ml.isna().sum().sum() / (df_final_ml.shape[0]*df_final_ml.shape[1]) * 100
print('Il y a donc ', pourcentage_missing_value,'% de valeurs manquantes dans le dataframe')

## 3ème temps : s'occuper de ces valeurs manquantes !

On utilise une class de scikit-learn récemment crée : "IterativeImputer"

Cette classe utilise les autres colonnes (features) pour remplacer les valeurs manquantes !

Dans notre cas, ceci a du sens car globalement, les variations de température sont relativement proches sur tout le territoire français.

### Librairies

In [ ]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

### Fonction d'imputation

In [ ]:
def impute_na(df):
    ### Création du nouveau dataframe avec les valeurs qui ont été remplacées
    imputer = IterativeImputer(random_state = 42)
    array_df = df_final_ml.values ### Converti le dataframe en tableau 
    new_array = imputer.fit_transform(array_df)
    df_sans_na = pd.DataFrame(new_array) 
    
    ### Mettre les dates et nom de colonnes de l'ancien sur le nouveau
    liste_col = list(df_final_ml.columns)
    liste_date = list(df_final_ml.index)
    df_sans_na.columns = liste_col
    df_sans_na['date'] = liste_date
    df_final = df_sans_na.set_index('date')
    
    return df_final

X = impute_na(df_final_ml)

### Vérification de la sortie

In [ ]:
print('Shape de X : ',X.shape)
print('_____________________')
X.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(X.isna(), cbar=False)

Ainsi, toutes les valeurs ont bien été remplacées comme voulu, le preprocessing est terminé !

# Création des labels (y)

Le dossier qui permettra d'avoir les labels est un excel de Eco2mix RTE, un site qui regroupe toutes les informations de productions, de consommation et d'échange électrique. 

Nous voulons avoir comme sortie les données de production éolienne. 

In [ ]:
df_rte = pd.read_excel('Projet-EDF/eCO2mix_RTE_2019-actuel (1).xlsx')
df_rte.head()

On remarque bien les colonnes "Solaire", ainsi il faut réaliser les tâches suivantes lors du pre-processing : 
   - sélectionner seulement les colonnes qui nous intéressent : date ; heure ; eolien ; solaire
   - regrouper la date et heure en une colonne de type "Datetime" (car Heures est de type object)
   - Fusionner le dataframe des données d'entraînements avec ce nouveau dataframe des labels sur les heures pour avoir la production électrique aux mêmes pas de temps que le X_train

### Preprocessing de y

In [ ]:
def preprocessing_rte(df):

    df = df.drop(df.tail(1).index, axis=0) # drop last 1 row

    ### on garde les colonnes suivantes 
    columns_to_keep = ['Date','Heures','Eolien']
    df = df[columns_to_keep]
    
    ### Création d'une unique colonne Datatime (on passe en str puis de nouveau en datetime : + simple et rapide)
    df['Date_str'] = df['Date'].astype(str)
    df['Heures_str'] = df['Heures'].astype(str)
    df['Datetime_str'] = df['Date_str'] +' '+ df['Heures_str']

    format_string = "%Y-%m-%d %H:%M:%S" 
    df['date'] = df['Datetime_str'].apply(lambda x : datetime.datetime.strptime(x, format_string)) ###strptime passe d'un format string en un datetime
    
    ### On enelève les colonnes maintenant inutiles
    df = df.drop(['Date','Heures','Heures_str','Datetime_str','Date_str'], axis=1)
    
    ### On renomme les colonnes
    list_col = ['eolien','date']
    df.columns = list_col

    return df

y = preprocessing_rte(df_rte)

In [ ]:
print('Shape de y : ', y.shape)
print('___________________\n')
display(y.info())
print('___________________\n')
y.head()

### Fusion des données d'entraînement et les labels. 
 Nous pourrions nous satsifaire du travail précedent et éviter la fusion (surtout si c'est pour spliter de nouveau après). Cependant, il est important de mettre sur le même pas de temps toutes les données. Ainsi, des données manquantes peuvent apparaître. 

In [ ]:
def merge_X_y(X,y):
    X = X.reset_index()
    
    Xy = pd.merge(X, y, how='left', on='date')
    
    Xy = Xy.set_index('date')
    
    return Xy

Xy = merge_X_y(X,y)

In [ ]:
print('Shape de Xy : ', Xy.shape)
print('___________________\n')
display(Xy.info())
print('___________________\n')
print('Nombre de valeurs manquantes : ', Xy.isna().sum().sum())
print('___________________\n')
Xy.head()

# Modèle

Maintenant l'objectif est de construire un algorithme de ML et/ou DL permettant le point suivant : 
- On donne à l'algorithme l'historique des données de vent des stations dans toutes la France sur les 24H précédents (entre J et J-1)
- L'algo doit prédire le facteur de charge de la production éolienne sur 24h APRÈS (J+1)

On doit alors mettre en forme ces données

## Mise en forme de X et y à partir de Xy , permettant d'alimenter l'algo de DL

In [ ]:
def mise_en_forme(Xy, pas = 24):
    m = len(Xy) ### Nombre de ligne (=exemples)
    
    X = []
    y = []
    
    for index_row in range(pas, m-pas):
        X_1_example = Xy.iloc[index_row - pas : index_row, :-2 ]
        X_ravel = np.ravel(X_1_example)
        X.append(X_ravel)
        
        y_1_example = Xy.iloc[index_row + pas, -2:].tolist()
        y.append(y_1_example)
        
    X_df = pd.DataFrame(X)
    y_df = pd.DataFrame(y)
    
    return [X_df, y_df]

X, y = mise_en_forme(Xy)

In [ ]:
print('Shape de X : ', X.shape)
print('___________________\n')
display(X.info())
print('___________________\n')
X.head()

In [ ]:
print('Shape de y : ', y.shape)
print('___________________\n')
display(y.info())
print('___________________\n')
y.head()

## Train/Val/Test set

Maintenant, on va séparer notre X et y en 3 split, permettant d'entraîner notre modèle sur le train set et l'améliorer sur la validation set pour enfin l'évaluer sur le test set

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def split(X, y, test_size = 0.2, val_size = 0.2):
    
    scaler = StandardScaler() 
    
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size = 0.2)

    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size = val_size)

    
    return [X_train_full, X_train, X_val, X_test, y_train_full, y_train , y_val, y_test]
    
X_train_full, X_train, X_val, X_test, y_train_full, y_train , y_val, y_test = split(X, y)

In [ ]:
print('X_train_full shape : ',X_train_full.shape)
print('X_train shape : ',X_train.shape)
print('X_val shape : ',X_val.shape)
print('X_test shape : ',X_test.shape)
print('y_train_full shape : ',y_train_full.shape)
print('y_train shape : ',y_train.shape)
print('y_val shape : ',y_val.shape)
print('y_test shape : ',y_test.shape)

## Sélection de modèle

### Librairies

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Modèle

On commence avec des modèles de regression simple pour abouter sur des réseaux de neurones complexes

In [ ]:
import lazypredict

In [ ]:
models

In [ ]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(verbose = 2, ignore_warnings = False, custom_metric =None)
models, predictions = reg.fit(X_train_full, X_test, y_train_full, y_test)

In [ ]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(verbose = 2, ignore_warnings = False, custom_metric =None)
models_w_scale, predictions_w_scale = reg.fit(X_train_full_scale, X_test_scale, y_train_full, y_test)

In [ ]:
models_w_scale

On remarque alors que, dans ce type de problème, une MLP semble être le meilleur algorithme. 
On va alors optimiser les hyperparamètre pour avoir le meilleur résultat possible avec le MLP Regressor de Sklearn !

## Optimisation du MLP Regressor

Dans un premier temps, on va optimiser les paramètres suivant du MLPRegressor : hidden_layer_size et alpha 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

In [ ]:
parameters = {'hidden_layer_sizes' : [(128, 64, 32, 16), (64, 32), (1024, )]}
reg = GridSearchCV(MLPRegressor(), parameters)

reg.fit(X_train_full_scale, y_train_full)

In [ ]:
print(reg.score(X_train_full, y_train_full))
print(reg.best_params_)

## Modèle de Deep Learning avec Keras

In [ ]:
def create_model_simple_sans_dropout():

    inputs = keras.Input(shape=(8208,))


    x = layers.Dense(1024, activation='relu',kernel_initializer="he_normal")(inputs)
    
    x = layers.Dense(256, activation='relu',kernel_initializer="he_normal")(x)
        
    x = layers.Dense(64, activation='relu',kernel_initializer="he_normal")(x)
    
    x = layers.Dense(8, activation='relu',kernel_initializer="he_normal")(x)

    outputs = layers.Dense(2)(x)
    
    model = keras.Model(inputs = inputs, outputs = outputs, name='model_simple_sans_dropout')
    
    return model

def create_model_simple_avec_dropout(dropout_rate = 0.2):

    inputs = keras.Input(shape=(8208,))


    x = layers.Dense(1024, activation='relu',kernel_initializer="he_normal")(inputs)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(256, activation='relu',kernel_initializer="he_normal")(x)
    x = layers.Dropout(dropout_rate)(x)
       
    x = layers.Dense(64, activation='relu',kernel_initializer="he_normal")(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(8, activation='relu',kernel_initializer="he_normal")(x)

    outputs = layers.Dense(2)(x)
    
    model = keras.Model(inputs = inputs, outputs = outputs, name='model_simple_avec_dropout')
    
    return model

def create_model_complexe_sans_dropout():

    inputs = keras.Input(shape=(8208,))

    x = layers.Dense(2028, activation='relu',kernel_initializer="he_normal")(inputs)

    x = layers.Dense(1024, activation='relu',kernel_initializer="he_normal")(x)

    x = layers.Dense(512, activation='relu',kernel_initializer="he_normal")(x)

    x = layers.Dense(256, activation='relu',kernel_initializer="he_normal")(x)

    x = layers.Dense(128, activation='relu',kernel_initializer="he_normal")(x)

    x = layers.Dense(64, activation='relu',kernel_initializer="he_normal")(x)
    
    x = layers.Dense(8, activation='relu',kernel_initializer="he_normal")(x)

    outputs = layers.Dense(2)(x)
    
    model = keras.Model(inputs = inputs, outputs = outputs, name='model_complexe_sans_dropout')
    
    return model

def create_model_complexe_avec_dropout(dropout_rate = 0.2):

    inputs = keras.Input(shape=(8208,))
    
    x = layers.Dense(2028, activation='relu',kernel_initializer="he_normal")(inputs)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(1024, activation='relu',kernel_initializer="he_normal")(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(512, activation='relu',kernel_initializer="he_normal")(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(256, activation='relu',kernel_initializer="he_normal")(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(128, activation='relu',kernel_initializer="he_normal")(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(8, activation='relu',kernel_initializer="he_normal")(x)


    outputs = layers.Dense(2)(x)
    
    model = keras.Model(inputs = inputs, outputs = outputs, name='model_complexe_avec_dropout')
    
    return model

In [ ]:
model1 = create_model_simple_sans_dropout()
model2 = create_model_simple_avec_dropout()
model3 = create_model_complexe_sans_dropout()
model4 = create_model_complexe_avec_dropout()

model1.summary()

In [ ]:
model1.compile(
    loss=keras.losses.MSE,
    optimizer=keras.optimizers.RMSprop(),
)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_best_model_simple_nodropout_v2.h5", save_best_only=True)

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model1.fit(X_train, y_train, batch_size=32, epochs=60, validation_data=(X_val, y_val), callbacks=[checkpoint_cb,tensorboard_cb])

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
y_pred_train = model1.predict(X_train)

In [ ]:
y_train - y_pred_train).mean()

In [ ]:
pd.DataFrame(y_test_pred)

In [ ]:
model_simple_nodrop = keras.models.load_model("my_model_EDF_complexe_avec_dropout.h5")

In [ ]:
y_pred_val = pd.DataFrame(model1.predict(X_val))
y_pred_val

In [ ]:
abs(y_val - y_pred_val).mean()

In [ ]:
y_val